In [ ]:
# ----------------------------
# 1. IMPORT LIBRARIES
# ----------------------------
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("TensorFlow version:", tf.__version__)

In [ ]:
# ----------------------------
# 2. LOAD AND PREPROCESS DATA
# ----------------------------
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize to [-1, 1]
x_train = (x_train.astype('float32') - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

print(" Data shape:", x_train.shape)
print(" Pixel range:", x_train.min(), "to", x_train.max())

In [ ]:
# ----------------------------
# 3. BUILD GENERATOR
# ----------------------------
def build_generator():
    model = models.Sequential([
        # First dense layer
        layers.Dense(7*7*256, input_dim=100),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        
        # Reshape to 7x7x256
        layers.Reshape((7, 7, 256)),
        
        # Upsample to 14x14
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        
        # Upsample to 28x28
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

generator = build_generator()
print(" Generator built:")
generator.summary()

In [ ]:
# ----------------------------
# 4. BUILD DISCRIMINATOR
# ----------------------------
def build_discriminator():
    model = models.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

discriminator = build_discriminator()
print(" Discriminator built:")
discriminator.summary()

In [ ]:
# ----------------------------
# 5. COMPILE MODELS
# ----------------------------
# Optimizer
opt = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Discriminator
discriminator.compile(
    optimizer=opt,
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

# GAN (combined model)
discriminator.trainable = False
gan_input = layers.Input(shape=(100,))
gan_output = discriminator(generator(gan_input))
gan = models.Model(gan_input, gan_output)
gan.compile(
    optimizer=opt,
    loss=losses.BinaryCrossentropy()
)

In [ ]:
# ----------------------------
# 6. TRAINING LOOP
# ----------------------------
EPOCHS = 20
BATCH_SIZE = 128
noise_dim = 100

# For visualization
fixed_noise = tf.random.normal([16, noise_dim])

# Track losses
gen_losses = []
disc_losses = []

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    epoch_gen_loss = 0
    epoch_disc_loss = 0
    num_batches = 0
    
    for i in tqdm(range(0, len(x_train), BATCH_SIZE)):
        # Get real batch
        real_batch = x_train[i:i+BATCH_SIZE]
        batch_size = real_batch.shape[0]
        
        # Train Discriminator
        noise = tf.random.normal([batch_size, noise_dim])
        fake_batch = generator(noise, training=False)
        
        real_labels = tf.ones((batch_size, 1))
        fake_labels = tf.zeros((batch_size, 1))
        
        # Combine real and fake
        all_images = tf.concat([real_batch, fake_batch], axis=0)
        all_labels = tf.concat([real_labels, fake_labels], axis=0)
        
        # Add label smoothing
        all_labels += 0.05 * tf.random.uniform(all_labels.shape)
        
        d_loss = discriminator.train_on_batch(all_images, all_labels)
        
        # Train Generator
        noise = tf.random.normal([batch_size, noise_dim])
        g_labels = tf.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, g_labels)
        
        epoch_gen_loss += g_loss
        epoch_disc_loss += d_loss[0]
        num_batches += 1
    
    # Average losses
    avg_g_loss = epoch_gen_loss / num_batches
    avg_d_loss = epoch_disc_loss / num_batches
    gen_losses.append(avg_g_loss)
    disc_losses.append(avg_d_loss)
    
    print(f"Generator Loss: {avg_g_loss:.4f}, Discriminator Loss: {avg_d_loss:.4f}")
    
    # Generate and save images every 5 epochs
    if (epoch + 1) % 5 == 0:
        generated = generator(fixed_noise, training=False)
        plt.figure(figsize=(8, 8))
        for i in range(16):
            plt.subplot(4, 4, i+1)
            plt.imshow(generated[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.axis('off')
        plt.suptitle(f'Generated Digits - Epoch {epoch+1}')
        plt.savefig(f'gan_epoch_{epoch+1}.png')
        plt.show()

In [ ]:
# ----------------------------
# 7. PLOT LOSS CURVES
# ----------------------------
plt.figure(figsize=(10, 6))
plt.plot(gen_losses, label='Generator Loss', color='blue')
plt.plot(disc_losses, label='Discriminator Loss', color='red')
plt.title('GAN Training Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('gan_loss_curve.png')
plt.show()

In [ ]:
# Generate final samples
final_noise = tf.random.normal([25, noise_dim])
final_generated = generator(final_noise, training=False)

plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(final_generated[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    plt.axis('off')
plt.suptitle('Final Generated Digits (25 samples)')
plt.savefig('gan_final_samples.png')
plt.show()